In [ ]:
%%capture
!git clone https://github.com/cuongtv312/marl-delivery.git
%cd marl-delivery
!pip install -r requirements.txt

In [ ]:
from env import Environment
import torch
from collections import deque
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import numpy as np
import random
import os
from sklearn.calibration import LabelEncoder
import copy
import matplotlib.pyplot as plt

# Hyperparams

In [ ]:
SEED = 2025

ACTION_DIM = 15
NUM_AGENTS = 10
MAP_FILE = "map5.txt"
N_PACKAGES = 1000

MOVE_COST = -0.01
DELIVERY_REWARD = 10
DELAY_REWARD = 1
MAX_TIME_STEPS = 1000

NUM_EPISODES = 1000
BATCH_SIZE = 64
GAMMA = 0.99
LR = 5e-5
WEIGHT_DECAY = 1e-4
MAX_REPLAY_BUFFER_SIZE = 10000
EPS_START = 1
EPS_END = 0.1
EPS_DECAY = 1000
TAU = 0.01
GRADIENT_CLIPPING = 10


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)


# Convert State


In [ ]:
def manhattan_distance(x1, x2, y1, y2):
    return abs(x2 - x1) + abs(y2 - y1)

def bfs_distance(map_grid, start_x, start_y, goal_x, goal_y): #0-indexed
    start = (start_x, start_y)
    goal = (goal_x, goal_y)
    
    n_rows = len(map_grid)
    n_cols = len(map_grid[0])
    queue = deque([(goal, 0)])
    visited = set([goal])

    while queue:
        current, dist = queue.popleft()
        if current == start:
            return dist

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            next_pos = (current[0] + dx, current[1] + dy)
            if 0 <= next_pos[0] < n_rows and 0 <= next_pos[1] < n_cols:
                if map_grid[next_pos[0]][next_pos[1]] == 0 and next_pos not in visited:
                    visited.add(next_pos)
                    queue.append((next_pos, dist + 1))

    return float('inf')  

In [ ]:
def convert_state(state, persistent_packages, current_robot_idx):
    """
    Convert robot's observation to a 2D multi-channel tensor.
    - 6 channels for robot-specific observation:
        0. Map
        1. Urgency of 'waiting' packages (if robot is not carrying)
        2. Start positions of 'waiting' packages (if robot is not carrying)
        3. Other robots' positions
        4. Current robot's position
        5. Current robot's carried package target (if robot is carrying)

    Args:
        state (dict): Raw state from the environment. Expected keys: "map", "robots", "time_step".
                      state["robots"] is a list of tuples: (x+1, y+1, carrying_package_id)
        prev_state(dict): previous state
        persistent_packages (dict): Dict tracking all active packages.
        current_robot_idx (int): Index of the current robot.

    Returns:
        np.ndarray of shape (6, n_rows, n_cols)
    """
    grid = np.array(state["map"])
    n_rows, n_cols = grid.shape
    n_channels = 6
    tensor = np.zeros((n_channels, n_rows, n_cols), dtype=np.float32)

    #Channel 0: Map
    tensor[0] = grid

    current_time_step = state["time_step"]
    if isinstance(current_time_step, np.ndarray): 
        current_time_step = current_time_step[0]

    if current_robot_idx < 0 or current_robot_idx >= len(state["robots"]):
        print("Invalid robot idx")
        return tensor 

    #current robot data
    current_robot_data = state["robots"][current_robot_idx]
    carried_pkg_id_by_current_robot = current_robot_data[2] 

    #Channel 1: Urgency of 'waiting' packages (if not carrying) 
    #Channel 2: Start positions of 'waiting' packages (if not carrying)
    if carried_pkg_id_by_current_robot == 0: # Robot is not carrying
        for pkg_id, pkg_data in persistent_packages.items():
            if pkg_data['status'] == 'waiting':
                # all 1-indexed
                rr, rc, _ = current_robot_data
                dr, dc = pkg_data['target_pos']
                sr, sc = pkg_data['start_pos'] 
                st = pkg_data['start_time']
                dl = pkg_data['deadline']

                travel_length = bfs_distance(grid, rr - 1, rc - 1, sr - 1, sc - 1) + bfs_distance(grid, sr - 1, sc - 1, dr - 1, dc - 1)
                time_left = dl - current_time_step - travel_length
                #check active package
                if current_time_step >= st:
                    urgency = 0
                    # If no time left then no urgency
                    if dl > st and time_left > 0:
                        urgency = 1.0/time_left
      

                    if 0 <= sr < n_rows and 0 <= sc < n_cols: 
                        tensor[1, sr, sc] = max(tensor[1, sr, sc], urgency) 

                    # Channel 2: Start position
                    if 0 <= sr < n_rows and 0 <= sc < n_cols: 
                        tensor[2, sr, sc] = 1.0 
    

    # Channel 3: Other robots' positions
    for i, rob_data in enumerate(state["robots"]):
        if i == current_robot_idx:
            continue 
        rr, rc, _ = rob_data 
        rr_idx, rc_idx = int(rr) - 1, int(rc) - 1 
        if 0 <= rr_idx < n_rows and 0 <= rc_idx < n_cols: 
            tensor[3, rr_idx, rc_idx] = 1.0

    #Channel 4: Current robot's position
    crr, crc, _ = current_robot_data 
    crr_idx, crc_idx = int(crr) - 1, int(crc) - 1 
    if 0 <= crr_idx < n_rows and 0 <= crc_idx < n_cols: 
        tensor[4, crr_idx, crc_idx] = 1.0

    #Channel 5: Current robot's carried package target (if robot is carrying)
    if carried_pkg_id_by_current_robot != 0:
        if carried_pkg_id_by_current_robot in persistent_packages:
            pkg_data_carried = persistent_packages[carried_pkg_id_by_current_robot]
            tr_carried, tc_carried = pkg_data_carried['target_pos'] 
            if 0 <= tr_carried < n_rows and 0 <= tc_carried < n_cols: 
                tensor[5, tr_carried, tc_carried] = 1.0

    return tensor

# DQNetwork


In [ ]:
class DQNetwork(nn.Module):
    def __init__(self, observation_shape, action_dim):
        super(DQNetwork, self).__init__()
        self.conv1 = nn.Conv2d(observation_shape[0], 16, kernel_size=3, stride=1, padding='same')
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same')
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding='same')
        self.bn3 = nn.BatchNorm2d(32)
        flat_size = 32 * observation_shape[1] * observation_shape[2]  

        self.fc1 = nn.Linear(flat_size, 128)
        self.fc2 = nn.Linear(128, action_dim)

    def forward(self, obs):
        # obs: (N, C, H, W) or (C, H, W)
        if obs.dim() == 3:
            obs = obs.unsqueeze(0)  # (1, C, H, W)
        x = F.relu(self.bn1(self.conv1(obs)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.flatten(start_dim=1)  # (N, 32*H*W)
        x = F.relu(self.fc1(x))
        q_values = self.fc2(x)
        return q_values

# Replay Buffer

In [ ]:
class ReplayBuffer:
    def __init__(self, capacity, obs_shape, device="cpu"):
        self.capacity = capacity
        self.obs_shape = obs_shape
        self.device = device

        _obs_s = (obs_shape,) if isinstance(obs_shape, int) else obs_shape

        self.observations = np.zeros((capacity, *_obs_s), dtype=np.float32)
        self.actions = np.zeros((capacity,), dtype=np.int64)
        self.rewards = np.zeros((capacity,), dtype=np.float32)
        self.next_observations = np.zeros((capacity, *_obs_s), dtype=np.float32)
        self.dones = np.zeros((capacity,), dtype=np.bool_)

        self.ptr = 0
        self.size = 0

    def add(self, obs, action, reward, next_obs, done):
        """
        Adds a transition to the buffer.
        - obs: np.array with shape self.obs_shape
        - action: int
        - reward: float
        - next_obs: np.array with shape self.obs_shape
        - done: bool
        """
        self.observations[self.ptr] = obs
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.next_observations[self.ptr] = next_obs
        self.dones[self.ptr] = done

        self.ptr = (self.ptr + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)

    def sample(self, batch_size):
        if self.size == 0:
            print("Buffer is empty. Returned empty tensors")
            _obs_s_runtime = (self.obs_shape,) if isinstance(self.obs_shape, int) else self.obs_shape
            empty_obs = torch.empty((0, *_obs_s_runtime), dtype=torch.float32, device=self.device)
            empty_actions = torch.empty((0,), dtype=torch.long, device=self.device)
            empty_rewards = torch.empty((0,), dtype=torch.float32, device=self.device)
            empty_next_obs = torch.empty((0, *_obs_s_runtime), dtype=torch.float32, device=self.device)
            empty_dones = torch.empty((0,), dtype=torch.float32, device=self.device)
            return (empty_obs, empty_actions, empty_rewards, empty_next_obs, empty_dones)

        if self.size < batch_size:
            print(f"Warning: Buffer size ({self.size}) is less than batch size ({batch_size}). Sampling all available data.")
            indices = np.arange(self.size)
        else:
            indices = np.random.choice(self.size, batch_size, replace=False)

        batch_obs = torch.tensor(self.observations[indices], dtype=torch.float32).to(self.device)
        batch_actions = torch.tensor(self.actions[indices], dtype=torch.long).to(self.device)
        batch_rewards = torch.tensor(self.rewards[indices], dtype=torch.float32).to(self.device)
        batch_next_obs = torch.tensor(self.next_observations[indices], dtype=torch.float32).to(self.device)
        batch_dones = torch.tensor(self.dones[indices], dtype=torch.float32).to(self.device)

        return (batch_obs, batch_actions, batch_rewards, batch_next_obs, batch_dones)

    def can_sample(self, batch_size):
        return self.size >= batch_size

    def __len__(self):
        return self.size

# Save model

In [ ]:
def save_model(policy_net, path="/kaggle/working/models/dqn_agent.pt"):
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    torch.save(policy_net.state_dict(), path)

# Reward Shaping

In [ ]:
SHAPING_SUCCESSFUL_PICKUP_BONUS = 1
SHAPING_SUCCESSFUL_DELIVERY_BONUS = 0 
SHAPING_WASTED_PICKUP_PENALTY = -1 # Tried to pick from an empty spot or while already carrying
SHAPING_WASTED_DROP_PENALTY = -1   # Tried to drop when not carrying

def reward_shaping(global_r, prev_env_state, current_env_state, actions_taken, persistent_packages_before_action, num_agents):
    """
    Shapes the global reward 'global_r' to produce individual rewards,
    using a snapshot of the persistent package tracker from before the action.

    Args:
        global_r (float): The global reward from the environment for the current step (s -> s').
        prev_env_state (dict): The raw environment state 's' before actions_taken.
                               Contains 'robots' (1-indexed pos) and 'time_step'.
        current_env_state (dict): The raw environment state 's'' after actions_taken.
                                  Contains 'robots' (1-indexed pos) and 'time_step'.
        actions_taken (list): List of actions [(move_idx, package_op_idx), ...]
        persistent_packages_before_action (dict): 
            A copy of `self.persistent_packages` before the current action.
            Structure: { pkg_id: {'id': ..., 'start_pos': (r,c), 'target_pos': (r,c), 
                                   'start_time': ..., 'deadline': ..., 
                                   'status': 'waiting'/'in_transit'}, ... }
            Positions are 0-indexed. 
        num_agents (int): The number of agents.

    Returns:
        list: A list of shaped rewards, one for each agent.
    """
    
    individual_rewards = [0.0] * num_agents

    for i in range(num_agents):
        individual_rewards[i] = global_r # Initialize with original reward

    current_time_from_env = current_env_state['time_step']

    if isinstance(current_time_from_env, np.ndarray):
        current_time_from_env = current_time_from_env[0]
    
    # prev_env_state might not have 'time_step'
    time_at_prev_state = prev_env_state.get('time_step', current_time_from_env -1)
    if isinstance(time_at_prev_state, np.ndarray):
        time_at_prev_state = time_at_prev_state[0]


    for i in range(num_agents):
        agent_action = actions_taken[i]
        package_op = agent_action[1]  # 0: None, 1: Pick, 2: Drop

        if i >= len(prev_env_state['robots']) or i >= len(current_env_state['robots']):
            continue

        prev_robot_info = prev_env_state['robots'][i]
        current_robot_info = current_env_state['robots'][i]

        robot_prev_pos_0idx = (prev_robot_info[0] - 1, prev_robot_info[1] - 1)
        robot_current_pos_0idx = (current_robot_info[0] - 1, current_robot_info[1] - 1)

        prev_carrying_id = prev_robot_info[2]
        current_carrying_id = current_robot_info[2]

        # 1. Shaping for PICKUP attempts
        if package_op == 1:
            if prev_carrying_id == 0 and current_carrying_id != 0:
                # Successfully picked up a package
                individual_rewards[i] += SHAPING_SUCCESSFUL_PICKUP_BONUS
            elif prev_carrying_id != 0:
                # Tried to pick up while already carrying
                individual_rewards[i] += SHAPING_WASTED_PICKUP_PENALTY
            elif prev_carrying_id == 0 and current_carrying_id == 0:
                # Attempted pickup but failed (still not carrying).
                # Check if a 'waiting' package was truly available at the robot's previous location.
                package_was_available_and_waiting = False
                for pkg_id, pkg_data in persistent_packages_before_action.items():
                    if pkg_data['status'] == 'waiting' and \
                       pkg_data['start_pos'] == robot_prev_pos_0idx and \
                       pkg_data['start_time'] <= time_at_prev_state: # Package must be active
                        package_was_available_and_waiting = True
                        break
                if not package_was_available_and_waiting:
                    individual_rewards[i] += SHAPING_WASTED_PICKUP_PENALTY

        # 2. Shaping for DROP attempts
        elif package_op == 2:
   

            
            if prev_carrying_id == 0:
                individual_rewards[i] += SHAPING_WASTED_DROP_PENALTY
                
    return individual_rewards


In [ ]:
env = Environment(map_file=MAP_FILE,
                  n_robots=NUM_AGENTS, 
                  n_packages=N_PACKAGES,
                  move_cost=MOVE_COST,
                  delivery_reward=DELIVERY_REWARD,
                  delay_reward=DELAY_REWARD,
                  seed=SEED,
                  max_time_steps=MAX_TIME_STEPS)
env.reset()

# Trainer

In [ ]:
class DQNTrainer:
    def __init__(self, env, lr=LR, weight_decay=WEIGHT_DECAY, gamma=GAMMA, tau=TAU, gradient_clipping=GRADIENT_CLIPPING):
        self.env = env
        OBS_DIM = (6, env.n_rows, env.n_cols ) 
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Initialize LabelEncoders for actions
        self.le_move = LabelEncoder()
        self.le_move.fit(['S', 'L', 'R', 'U', 'D']) # Stay, Left, Right, Up, Down
        self.le_pkg_op = LabelEncoder()
        self.le_pkg_op.fit(['0', '1', '2']) # 0: None, 1: Pickup, 2: Drop
        self.NUM_MOVE_ACTIONS = len(self.le_move.classes_) # 5
        self.NUM_PKG_OPS = len(self.le_pkg_op.classes_) # 3
        
        # Network
        self.agent_network = DQNetwork(OBS_DIM, ACTION_DIM).to(self.device)
        
        # Target networks
        self.target_agent_network = DQNetwork(OBS_DIM, ACTION_DIM).to(self.device)
        self.target_agent_network.load_state_dict(self.agent_network.state_dict())
        
        self.buffer = ReplayBuffer(capacity=MAX_REPLAY_BUFFER_SIZE, obs_shape=OBS_DIM, device=self.device)
        self.lr = lr
        self.weight_decay = weight_decay
        self.gamma = gamma
        self.tau = tau
        self.gradient_clipping = gradient_clipping
        
        # Persistent packages to track the package id and the target position
        self.persistent_packages = {}

        self.update_targets(1.0)  # Hard update at start

        self.agent_optimizer = optim.Adam(self.agent_network.parameters(), lr=lr, weight_decay=weight_decay)
        
    def _update_persistent_packages(self, current_env_state): 
        """
        Updates self.persistent_packages based on the current environment state.
        - current_env_state: The state dictionary from env.step() or env.reset().
        """
        # 1. Add newly appeared packages to persistent_packages if not already tracked
        if 'packages' in current_env_state and current_env_state['packages'] is not None:
            for pkg_tuple in current_env_state['packages']:
                pkg_id = pkg_tuple[0]
                if pkg_id not in self.persistent_packages:
                    self.persistent_packages[pkg_id] = {
                        'id': pkg_id,
                        'start_pos': (pkg_tuple[1] - 1, pkg_tuple[2] - 1),
                        'target_pos': (pkg_tuple[3] - 1, pkg_tuple[4] - 1),
                        'start_time': pkg_tuple[5],
                        'deadline': pkg_tuple[6],
                        'status': 'waiting'
                    }

        # 2. Get current robot carrying info
        current_carried_pkg_ids_set = set()
        if 'robots' in current_env_state and current_env_state['robots'] is not None:
            for r_idx, r_data in enumerate(current_env_state['robots']):
                carried_id = r_data[2] # (pos_x+1, pos_y+1, carrying_package_id)
                if carried_id != 0:
                    current_carried_pkg_ids_set.add(carried_id)

        packages_to_remove_definitively = []

        # 3. Update package status
        for pkg_id, pkg_data in list(self.persistent_packages.items()):
            original_status_in_tracker = pkg_data['status']

            if pkg_id in current_carried_pkg_ids_set:
                # If currently being carried by any robot in current_env_state, set to 'in_transit'
                self.persistent_packages[pkg_id]['status'] = 'in_transit'
            else:
                # Package is NOT being carried in current_env_state
                if original_status_in_tracker == 'in_transit':
                    # This package WAS 'in_transit' (according to our tracker)
                    # and is now NOT carried in current_env_state.
                    # Given the env.py logic, this means it MUST have been delivered correctly.
                    packages_to_remove_definitively.append(pkg_id)
                # If original_status_in_tracker was 'waiting' and it's still not carried,
                # its status remains 'waiting'. No change needed to start_pos or status here.
                pass

        # 4. Remove packages that were successfully delivered
        for pkg_id_to_remove in packages_to_remove_definitively:
            if pkg_id_to_remove in self.persistent_packages:
                del self.persistent_packages[pkg_id_to_remove]


    def update_targets(self, tau=None):
        if tau is None:
            tau = self.tau
        # Soft update
        for target_param, param in zip(self.target_agent_network.parameters(), self.agent_network.parameters()):
            target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)


    def select_action(self, obs, eps):
        # obs_batch: (C, H, W)
        if obs is not isinstance(obs, torch.Tensor) and isinstance(obs, np.ndarray):
            obs = torch.from_numpy(obs).float().to(self.device)
        if np.random.rand() < eps:
            action = np.random.randint(0, ACTION_DIM)
        else:
            with torch.no_grad():
                q_values = self.agent_network(obs)
                action = torch.argmax(q_values, dim=1).item()
        return action

    def train_step(self, batch_size):
        if not self.buffer.can_sample(batch_size):
            return None

        batch_obs, batch_actions, batch_rewards, batch_next_obs, batch_dones = self.buffer.sample(batch_size)
        # batch_obs: (B, C, H, W)
        # batch_actions: (B,)
        # batch_rewards: (B,)
        # batch_next_obs: (B, C, H, W)
        # batch_dones: (B,)

        # Compute Q(s, a) for the actions taken
        q_values = self.agent_network(batch_obs)  # (B, action_dim)
        q_value = q_values.gather(1, batch_actions.unsqueeze(1)).squeeze(1)  # (B,)

        # Compute target Q values
        with torch.no_grad():
            next_q_values = self.target_agent_network(batch_next_obs)  # (B, action_dim)
            next_q_value = next_q_values.max(1)[0]  # (B,)
            target = batch_rewards + self.gamma * (1 - batch_dones) * next_q_value

        # Loss
        loss = F.mse_loss(q_value, target)

        # Optimize
        self.agent_optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.agent_network.parameters(), self.gradient_clipping)
        self.agent_optimizer.step()

        # Soft update target network
        self.update_targets(self.tau)

        return loss.item()


    def run_episode(self, eps):
        current_state_dict = self.env.reset()
        self.persistent_packages = {}
        self._update_persistent_packages(current_state_dict)
        done = False
        episode_reward = 0
        episode_loss = 0
        step_count = 0

        while not done:
            # self.env.render_pygame()
            # Build per-agent observations
            actions = []
            observations = []
            env_actions = []
            packages_before_action = copy.deepcopy(self.persistent_packages)
            
            for i in range(NUM_AGENTS):
                obs = convert_state(current_state_dict, self.persistent_packages, current_robot_idx=i)
                observations.append(obs)
                action = self.select_action(obs, eps)
                actions.append(action)
            next_observations = []
            prev_state_dict = current_state_dict
            
            # Take actions and get next state
            for int_act in actions:
                move_idx = int_act % self.NUM_MOVE_ACTIONS
                pkg_op_idx = int_act // self.NUM_MOVE_ACTIONS

                # Ensure pkg_op_idx is within bounds
                if pkg_op_idx >= self.NUM_PKG_OPS:
                    print(f"Warning: Decoded pkg_op_idx {pkg_op_idx} is out of bounds for action {int_act}. Max is {self.NUM_PKG_OPS-1}. Defaulting to op index 0.")
                    pkg_op_idx = 0 # Default to the first package operation (e.g., 'None')
                
                move_str = self.le_move.inverse_transform([move_idx])[0]
                pkg_op_str = self.le_pkg_op.inverse_transform([pkg_op_idx])[0]
                env_actions.append((move_str, pkg_op_str))
                
                
            current_state_dict, global_reward, done, _= self.env.step(env_actions)
            individual_rewards = reward_shaping(global_reward, 
                                                prev_state_dict, 
                                                current_state_dict, 
                                                env_actions,
                                                packages_before_action,
                                                NUM_AGENTS)
            self._update_persistent_packages(current_state_dict)
            
            # Build per-agent next observations
            for i in range(NUM_AGENTS):
                next_obs = convert_state(current_state_dict, self.persistent_packages, current_robot_idx=i)
                next_observations.append(next_obs)

            # Store in buffer
            for i in range(NUM_AGENTS):
                self.buffer.add(
                    obs=observations[i],
                    action=actions[i],
                    reward=individual_rewards[i],
                    next_obs=next_observations[i],
                    done=done
                )

            episode_reward += global_reward
            step_count += 1

            # Training step
            loss = self.train_step(BATCH_SIZE)
            if loss is not None:
                episode_loss += loss


        return episode_reward, episode_loss / max(1, step_count)


# Training

In [ ]:
trainer = DQNTrainer(env)
def linear_epsilon(steps_done):
    return max(EPS_END, EPS_START - (EPS_START - EPS_END) * (steps_done / EPS_DECAY))


def exponential_epsilon(steps_done):
    return EPS_END + (EPS_START - EPS_END) * np.exp(-steps_done / EPS_DECAY)

steps_done = np.arange(NUM_EPISODES)

linear_epsilons = [linear_epsilon(step) for step in steps_done]
exp_epsilons = [exponential_epsilon(step) for step in steps_done]

# Lists to store metrics for plotting
episode_rewards_history = []
episode_avg_loss_history = []

training_completed_successfully = False
print("Starting DQN training...")
print(f"Running for {NUM_EPISODES} episodes.")

try:
    best_reward = -999
    for episode_num in range(1, NUM_EPISODES + 1):
        # The exponential_epsilon function from in[16] expects 'steps_done'
        # Assuming 'steps_done' in that context refers to the number of episodes completed (0-indexed)
        current_epsilon = linear_epsilon(episode_num - 1) 
        
        episode_reward, avg_episode_loss = trainer.run_episode(current_epsilon)
        
        episode_rewards_history.append(episode_reward)
        episode_avg_loss_history.append(avg_episode_loss)
        if episode_reward > best_reward:
            best_reward = episode_reward
            print(f"Save best model at episode {episode_num} with best_reward {best_reward}")
            save_model(trainer.agent_network, path=f"/kaggle/working/models/dqn_agent_best_reward.pt")
                
        if episode_num % 10 == 0 or episode_num == NUM_EPISODES: # Print every 10 episodes and the last one
            print(f"Episode {episode_num}/{NUM_EPISODES} | Reward: {episode_reward:.2f} | Avg Loss: {avg_episode_loss:.4f} | Epsilon: {current_epsilon:.3f}")

        # Optional: Periodic saving during training
        if episode_num % 50 == 0: # Example: Save every 50 episodes
            print(f"Saving checkpoint at episode {episode_num}...")
            save_model(trainer.agent_network, path=f"/kaggle/working/models/dqn_agent_ep{episode_num}.pt")
            
    training_completed_successfully = True

except KeyboardInterrupt:
    print("\nTraining interrupted by user (KeyboardInterrupt).")
    print("Saving current model state...")
    save_model(trainer.agent_network, path="/kaggle/working/models/dqn_agent_interrupted.pt")
    print("Models saved to _interrupted.pt files.")
except Exception as e:
    print(f"\nAn error occurred during training: {e}")
    import traceback
    traceback.print_exc() # Print detailed traceback for the exception
    print("Saving model due to exception...")
    save_model(trainer.agent_network, path="/kaggle/working/models/dqn_agent_exception.pt")
    print("Models saved to _exception.pt files.")
finally:
    # pygame.quit()
    print("\nTraining loop finished or was interrupted.")
    
    # Plotting the results
    if episode_rewards_history: # Check if there's any data to plot
        plt.figure(figsize=(14, 6))

        plt.subplot(1, 2, 1)
        plt.plot(episode_rewards_history)
        plt.title('Total Reward per Episode')
        plt.xlabel('Episode')
        plt.ylabel('Total Reward')
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(episode_avg_loss_history)
        plt.title('Average Loss per Episode')
        plt.xlabel('Episode')
        plt.ylabel('Average Loss')  
        plt.grid(True)

        plt.tight_layout()
        plt.show()
    else:
        print("No data recorded for plotting.")

if training_completed_successfully:
    print("\nTraining completed successfully.")
    print("Saving final model...")
    save_model(trainer.agent_network, path="/kaggle/working/models/dqn_agent_final.pt")
    print("Final models saved.")